In [2]:
from google.colab import files
uploaded = files.upload()


Saving GlobalLandTemperaturesByCountry.csv to GlobalLandTemperaturesByCountry.csv


In [3]:
import pandas as pd

# Replace 'filename.csv' with the name of your uploaded file
df = pd.read_csv('GlobalLandTemperaturesByCountry.csv')

# Display the first few rows and column names
print(df.head())
print(df.columns)


           dt  AverageTemperature  AverageTemperatureUncertainty Country
0  1743-11-01               4.384                          2.294   Åland
1  1743-12-01                 NaN                            NaN   Åland
2  1744-01-01                 NaN                            NaN   Åland
3  1744-02-01                 NaN                            NaN   Åland
4  1744-03-01                 NaN                            NaN   Åland
Index(['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'Country'], dtype='object')


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Load the dataset
df = pd.read_csv('GlobalLandTemperaturesByCountry.csv')

# Data preprocessing
df['dt'] = pd.to_datetime(df['dt'])
df['Year'] = df['dt'].dt.year
df['Month'] = df['dt'].dt.month_name()
df = df.dropna(subset=['AverageTemperature', 'Country'])

# Create widgets
country_dropdown = widgets.Dropdown(
    options=df['Country'].unique(),
    value='United States',
    description='Country:'
)

year_slider = widgets.IntRangeSlider(
    value=[1900, 2013],
    min=1900,
    max=2013,
    step=1,
    description='Year Range:',
    continuous_update=False
)

# Plot functions
def plot_temp(country, year_range, ax):
    filtered_df = df[(df['Country'] == country) &
                     (df['Year'] >= year_range[0]) &
                     (df['Year'] <= year_range[1])]
    if not filtered_df.empty:
        ax.plot(filtered_df['dt'], filtered_df['AverageTemperature'], label='Avg Temp', color='blue')
    ax.set_title(f'Average Temperature in {country} ({year_range[0]} - {year_range[1]})', fontsize=12)
    ax.set_xlabel('Year', fontsize=10)
    ax.set_ylabel('Average Temperature', fontsize=10)
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.7)

def plot_temp_distribution_by_month(country, year_range, ax):
    filtered_df = df[(df['Country'] == country) &
                     (df['Year'] >= year_range[0]) &
                     (df['Year'] <= year_range[1])]
    if not filtered_df.empty:
        filtered_df.boxplot(column='AverageTemperature', by='Month', grid=False, ax=ax)
    ax.set_title(f'Temperature Distribution by Month in {country} ({year_range[0]} - {year_range[1]})', fontsize=12)
    ax.set_xlabel('Month', fontsize=10)
    ax.set_ylabel('Average Temperature', fontsize=10)

def plot_avg_temp_by_year(country, year_range, ax):
    filtered_df = df[(df['Country'] == country) &
                     (df['Year'] >= year_range[0]) &
                     (df['Year'] <= year_range[1])]
    if not filtered_df.empty:
        yearly_avg = filtered_df.groupby('Year')['AverageTemperature'].mean().reset_index()
        ax.plot(yearly_avg['Year'], yearly_avg['AverageTemperature'], marker='o', color='green')
    ax.set_title(f'Average Temperature by Year in {country} ({year_range[0]} - {year_range[1]})', fontsize=12)
    ax.set_xlabel('Year', fontsize=10)
    ax.set_ylabel('Average Temperature', fontsize=10)
    ax.grid(True, linestyle='--', alpha=0.7)

def plot_temp_anomalies(country, year_range, ax):
    overall_avg_temp = df.groupby('Year')['AverageTemperature'].mean().mean()
    filtered_df = df[(df['Country'] == country) &
                     (df['Year'] >= year_range[0]) &
                     (df['Year'] <= year_range[1])]
    if not filtered_df.empty:
        yearly_avg = filtered_df.groupby('Year')['AverageTemperature'].mean().reset_index()
        yearly_avg['Anomaly'] = yearly_avg['AverageTemperature'] - overall_avg_temp
        ax.bar(yearly_avg['Year'], yearly_avg['Anomaly'], color='red', alpha=0.6)
    ax.axhline(0, color='black', linewidth=0.5)
    ax.set_title(f'Temperature Anomalies in {country} ({year_range[0]} - {year_range[1]})', fontsize=12)
    ax.set_xlabel('Year', fontsize=10)
    ax.set_ylabel('Temperature Anomaly (°C)', fontsize=10)
    ax.grid(True, linestyle='--', alpha=0.7)

# Alternate chart for Monthly temperature trend over time
def plot_monthly_temp_trend(country, year_range, ax):
    filtered_df = df[(df['Country'] == country) &
                     (df['Year'] >= year_range[0]) &
                     (df['Year'] <= year_range[1])]
    if not filtered_df.empty:
        monthly_avg = filtered_df.groupby(['Year', 'Month'])['AverageTemperature'].mean().unstack()
        monthly_avg.plot(ax=ax, legend=False, colormap='coolwarm')
    ax.set_title(f'Monthly Temperature Trend in {country} ({year_range[0]} - {year_range[1]})', fontsize=12)
    ax.set_xlabel('Year', fontsize=10)
    ax.set_ylabel('Temperature', fontsize=10)
    ax.grid(True, linestyle='--', alpha=0.7)

# Create Dashboard
def interactive_dashboard(country, year_range):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Plot 1: Average Temperature Over Time
    plot_temp(country, year_range, axes[0, 0])

    # Plot 2: Temperature Distribution by Month
    plot_temp_distribution_by_month(country, year_range, axes[0, 1])

    # Plot 3: Average Temperature by Year
    plot_avg_temp_by_year(country, year_range, axes[1, 0])

    # Plot 4: Monthly Temperature Trend Over Time (New)
    plot_monthly_temp_trend(country, year_range, axes[1, 1])

    plt.tight_layout(pad=3.0)
    plt.show()

# Create the interactive dashboard
widgets.interactive(interactive_dashboard, country=country_dropdown, year_range=year_slider)


interactive(children=(Dropdown(description='Country:', index=232, options=('Åland', 'Afghanistan', 'Africa', '…